In [1]:
!pip install pandas

In [2]:
import pandas as pd

In [6]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 808.1 kB/s eta 0:00:0000:0100:01


In [38]:
import pandas as pd
df=pd.read_csv('/Users/mankirat/Desktop/ml/ dataset_1 985 rows.csv')
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib
import requests
import json
features = [
    'avg_loss_making_trades',
    'avg_profitable_trades',
    'collection_score',
    'diamond_hands',
    'fear_and_greed_index',
    'holder_metrics_score',
    'liquidity_score',
    'loss_making_trades',
    'loss_making_trades_percentage',
    'loss_making_volume',
    'market_dominance_score',
    'metadata_score',
    'profitable_trades',
    'profitable_trades_percentage',
    'profitable_volume',
    'token_distribution_score',
    'washtrade_index'
]

X=df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
clf = IsolationForest(
    contamination=0.1,  # Assuming 10% of collections might be suspicious
    random_state=42,
    n_estimators=100
)
clf.fit(X_scaled)
joblib.dump(clf, 'isolation_forest_model.joblib')
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [47]:
def calculate_risk_score(data: pd.DataFrame, model, scaler) -> Tuple[float, str]:
    # Get anomaly scores
    score = model.score_samples(scaler.transform(data))
    
    # Add small epsilon to prevent division by zero
    epsilon = 1e-10
    
    # Handle edge case where min and max scores are equal
    score_range = score.max() - score.min()
    if abs(score_range) < epsilon:
        risk_score = 50.0  # default mid-range risk when all scores are identical
    else:
        # Normalize scores to 0-100 range with epsilon
        risk_score = 100 * (1 - (score - score.min()) / (score_range + epsilon))
    
    # Determine risk category
    if risk_score < 30:
        category = "Low Risk"
    elif risk_score < 70:
        category = "Medium Risk"
    else:
        category = "High Risk"
        
    return float(risk_score), category

In [48]:
# def predict_risk(new_contract_data):
#     # Load saved models
#     clf = joblib.load('isolation_forest_model.joblib')
#     scaler = joblib.load('scaler.joblib')
    
#     # Ensure new_contract_data has the same features in the same order
#     new_data = pd.DataFrame([new_contract_data], columns=features)
    
#     # Calculate risk score and category
#     risk_score, risk_category = calculate_risk_score(new_data, clf, scaler)
    
#     return {
#         'risk_score': round(risk_score, 2),
#         'risk_category': risk_category,
#         'contributing_factors': identify_risk_factors(new_data)
#     }
def predict_risk(contract_dict):
    try:
        # Load saved models
        clf = joblib.load('isolation_forest_model.joblib')
        scaler = joblib.load('scaler.joblib')
        
        # Convert dictionary to DataFrame with proper structure
        df_data = pd.DataFrame([contract_dict])
        
        # Ensure all required features are present
        if not all(feature in df_data.columns for feature in features):
            missing = set(features) - set(df_data.columns)
            raise ValueError(f"Missing required features: {missing}")
            
        # Reorder columns to match model features
        new_data = df_data[features]
        
        # Calculate risk score and category
        risk_score, risk_category = calculate_risk_score(new_data, clf, scaler)
        
        return {
            'risk_score': round(risk_score, 2),
            'risk_category': risk_category,
            'contributing_factors': identify_risk_factors(new_data)
        }
        
    except Exception as e:
        print(f"Error in risk prediction: {str(e)}")
        return None


In [49]:
def identify_risk_factors(data):
    risk_factors = []
    
    # Define thresholds for various metrics
    if data['washtrade_index'].iloc[0] > 50:
        risk_factors.append('High wash trading activity')
    if data['loss_making_trades_percentage'].iloc[0] > 70:
        risk_factors.append('High percentage of loss-making trades')
    if data['liquidity_score'].iloc[0] < 30:
        risk_factors.append('Low liquidity')
    if data['holder_metrics_score'].iloc[0] < 40:
        risk_factors.append('Poor holder metrics')
    
    return risk_factors

In [52]:
def riskify(contract_address):
    url = f"https://api.unleashnfts.com/api/v2/nft/collection/profile?blockchain=ethereum&contract_address={contract_address}&offset=0&limit=100&sort_by=washtrade_index&time_range=all&sort_order=desc"
    headers = {
            "accept": "application/json",
            "x-api-key": "19a1634dc850e33607b074bc62da2e19"
        }

        # Make the API request
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)['data']
    if data is None:
        print('data not available')
    else:
        print(predict_risk(data[0]))


In [53]:
riskify('0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d')

{'risk_score': 50.0, 'risk_category': 'Medium Risk', 'contributing_factors': ['High wash trading activity']}
